# W9 Prac - MLPS and Convolutional Neural Networks (CNNs) using PyTorch

In [6]:
!pip install torch

   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   - -------------------------------------- 5.5/212.5 MB 25.7 MB/s eta 0:00:09
   - -------------------------------------- 10.5/212.5 MB 27.3 MB/s eta 0:00:08
   -- ------------------------------------- 14.7/212.5 MB 23.6 MB/s eta 0:00:09
   --- ------------------------------------ 19.1/212.5 MB 23.2 MB/s eta 0:00:09
   ---- ----------------------------------- 25.7/212.5 MB 24.6 MB/s eta 0:00:08
   ------ --------------------------------- 32.2/212.5 MB 25.9 MB/s eta 0:00:07
   ------- -------------------------------- 39.1/212.5 MB 27.0 MB/s eta 0:00:07
   -------- ------------------------------- 46.4/212.5 MB 27.8 MB/s eta 0:00:06
   --------- ------------------------------ 53.0/212.5 MB 28.3 MB/s eta 0:00:06
   ----------- ---------------------------- 60.3/212.5 MB 29.1 MB/s eta 0:00:06
   ------------ --------------------------- 67.9/212.5 MB 29.6 MB/s eta 0:00:05
   -------------- ------------------------- 75.0/2

In [11]:
!pip install torchsummary

In [15]:
!pip install torchvision

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 15.5 MB/s eta 0:00:00


In [19]:
!pip install tensorboard

   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ------------------------ --------------- 3.4/5.5 MB 22.3 MB/s eta 0:00:01
   ---------------------------------------- 5.5/5.5 MB 18.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ---------------------------------------- 4.3/4.3 MB 28.6 MB/s eta 0:00:00


In [21]:
import torch
import torch.nn as nn   # Building blocks for neural networks
import torch.nn.functional as F # Various functions for building neural networks
import torch.optim as optim  # Optimiser for neural networks
from torchsummary import summary  # Summarise PyTorch model
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter # Tensorboard in PyTorch
import datetime
!rm -rf ./logs/
print(torch.__version__) # Double check the colab has the instance of tensorflow we want

# If using GPU
print('Cuda Available : {}'.format(torch.cuda.is_available()))
if torch.cuda.is_available():
  print('GPU - {0}'.format(torch.cuda.get_device_name()))

# Library for Progres Bar during training
!pip install tqdm
!pip install tensorboard

from tqdm import tqdm # Progress bar during training

2.7.0+cpu
Cuda Available : False


'rm' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image to tensor
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize the pixel values
])

# Load MNIST training dataset with transformations
mnist_train = datasets.MNIST('data', train=True, download=True, transform=transform)
mnist_test = datasets.MNIST('data', train=False, download=True, transform=transform)

100%|██████████| 9.91M/9.91M [00:08<00:00, 1.20MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 113kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 916kB/s] 
100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]


In [25]:
# Split dataset into training, validation, and test sets
train_size = int(0.8 * len(mnist_train))
val_size = len(mnist_train) - train_size
mnist_train, mnist_val = random_split(mnist_train, [train_size, val_size])

# Define data loaders
train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
val_loader = DataLoader(mnist_val, batch_size=64)
test_loader = DataLoader(mnist_test, batch_size=64)

In [27]:
class MLP(nn.Module):
  def __init__(self):
      super(MLP, self).__init__()
      self.flatten = nn.Flatten()
      self.fc1 = nn.Linear(28*28, 512)
      self.fc2 = nn.Linear(512, 10)

  def forward(self, x):
      x = self.flatten(x)
      x = F.relu(self.fc1(x))
      x = F.softmax(self.fc2(x), dim=1)
      return x

In [29]:
class MLP(nn.Module):
  def __init__(self):
      super(MLP, self).__init__()
      self.model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.Softmax(dim=1)
    )

  def forward(self, x):
      x = self.model(x)
      return x
     

In [31]:
# Instantiate MLP
model = MLP().cuda() if torch.cuda.is_available() else MLP()

# Optimiser
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Loss function
loss_fn = nn.CrossEntropyLoss()

# Set up TensorBoard log directory
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/MLP"
writer = SummaryWriter(log_dir)

# Example training loop
num_epochs = 5 # Vary as you wish

In [35]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        # When cuda is available
        if torch.cuda.is_available():
          inputs = inputs.cuda()
          labels = labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    # Logging training loss and accuracy
    writer.add_scalar('Loss/train', running_loss / len(train_loader), epoch)
    writer.add_scalar('Accuracy/train', 100. * correct / total, epoch)
    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:  # assume you have DataLoader for test_ds
            # When cuda is available
            if torch.cuda.is_available():
              inputs = inputs.cuda()
              labels = labels.cuda()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    # Logging validation loss and accuracy
    writer.add_scalar('Loss/val', val_loss / len(val_loader), epoch)
    writer.add_scalar('Accuracy/val', 100. * val_correct / val_total, epoch)

# Don't forget to close the writer when done
writer.close()
print("--------------")
# Summarise the model
summary(model, input_size=(1, 28*28))

Epoch 5: 100%|██████████| 750/750 [00:31<00:00, 23.71it/s]


--------------
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 512]         401,920
              ReLU-3                  [-1, 512]               0
            Linear-4                   [-1, 10]           5,130
           Softmax-5                   [-1, 10]               0
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 1.55
Estimated Total Size (MB): 1.57
----------------------------------------------------------------


In [37]:
%load_ext tensorboard
%tensorboard --logdir ./logs/fit

In [39]:
# Set up TensorBoard writer
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/CNN"
writer = SummaryWriter(log_dir)

# Define CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)  # MNIST has 1 color (channel)
        self.fc1 = nn.Linear(32 * 26 * 26, 512)  # Flatten after Conv2D
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model (using cuda if it is available)
model = CNN().cuda() if torch.cuda.is_available() else CNN()

# Define optimizer and loss function
optimizer = optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

# Training loop
num_epochs = 5  # Vary as you wish
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        # When cuda is available
        if torch.cuda.is_available():
          inputs = inputs.cuda()
          labels = labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    # Log loss and accuracy to TensorBoard
    writer.add_scalar('Loss/train', running_loss / len(train_loader), epoch)
    writer.add_scalar('Accuracy/train', 100. * correct / total, epoch)

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            # When cuda is available
            if torch.cuda.is_available():
              inputs = inputs.cuda()
              labels = labels.cuda()

            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    # Log validation loss and accuracy
    writer.add_scalar('Loss/val', val_loss / len(test_loader), epoch)
    writer.add_scalar('Accuracy/val', 100. * val_correct / val_total, epoch)

# Don't forget to close the writer
writer.close()
print("--------------")

# Summarise the model
summary(model, input_size=(1, 28, 28))



Epoch 5: 100%|██████████| 750/750 [03:15<00:00,  3.85it/s]


--------------
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
            Linear-2                  [-1, 512]      11,076,096
            Linear-3                   [-1, 10]           5,130
Total params: 11,081,546
Trainable params: 11,081,546
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.17
Params size (MB): 42.27
Estimated Total Size (MB): 42.44
----------------------------------------------------------------


In [41]:
%reload_ext tensorboard
%tensorboard --logdir ./logs/fit

Reusing TensorBoard on port 6006 (pid 81140), started 0:29:15 ago. (Use '!kill 81140' to kill it.)